# CLIP Japanese v2 画像・動画検索エンジン

## 概要

`line-corporation/clip-japanese-base-v2` を使った画像・動画の類似検索エンジンを試すノートブック。

### できること
- **画像** → 類似画像・動画を検索
- **動画** → 類似画像・動画を検索（フレーム抽出→平均ベクトル化）
- **テキスト（日本語対応）** → 類似画像・動画を検索
- **ディレクトリ一括登録**（サブディレクトリ再帰対応）

### 使い方
1. 下の「設定」セルでパスを自分の環境に合わせて変更する
2. 上から順にセルを実行する

## 1. セットアップ

In [ ]:
import sys
import os
from pathlib import Path

# clip-search-engine モジュールをインポートできるようにする
ENGINE_DIR = Path(os.getcwd()).parent / "clip-search-engine"
sys.path.insert(0, str(ENGINE_DIR))

print(f"エンジンディレクトリ: {ENGINE_DIR}")
print(f"存在確認: {ENGINE_DIR.exists()}")

## 2. 設定（★ここを自分の環境に合わせて変更する★）

以下のパスをプレースホルダーから実際のパスに書き換えてください。

In [ ]:
# =====================================
# ★ 設定: 以下のパスを自分の環境に書き換える
# =====================================

# 画像・動画が入っているディレクトリ（DBに登録する対象）
# サブディレクトリも自動的に再帰探索される
MEDIA_DIR = Path(r"YOUR_MEDIA_DIR_PATH_HERE")  # 例: Path(r"D:\写真\旅行2024")

# 検索クエリに使う画像のパス（任意）
QUERY_IMAGE_PATH = Path(r"YOUR_QUERY_IMAGE_PATH_HERE")  # 例: Path(r"D:\写真\query.jpg")

# 検索クエリに使う動画のパス（任意）
QUERY_VIDEO_PATH = Path(r"YOUR_QUERY_VIDEO_PATH_HERE")  # 例: Path(r"D:\動画\query.mp4")

# ベクトルDBの保存先（デフォルトのままでOK）
DB_DIR = ENGINE_DIR / "db"

# モデルキャッシュの保存先（既にダウンロード済みのパスがあればそれを指定）
CACHE_DIR = ENGINE_DIR / "model"
# 例: 既存のキャッシュを再利用する場合:
# CACHE_DIR = Path(r"D:\ドキュメント\code\clip-japanese-v2-sample\model")

# 動画のフレーム抽出間隔（秒）
FRAME_INTERVAL_SEC = 10.0

# テキスト検索クエリ
TEXT_QUERY = "猫が寝ている"

print("設定完了")
print(f"  メディアディレクトリ: {MEDIA_DIR}")
print(f"  DB保存先: {DB_DIR}")
print(f"  モデルキャッシュ: {CACHE_DIR}")

## 3. モデルのロード

初回実行時はモデルのダウンロードに数分かかります。
2回目以降は `CACHE_DIR` からロードされるため高速です。

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

from clip_model import CLIPJapaneseModel

model = CLIPJapaneseModel(cache_dir=CACHE_DIR)
print(f"\nモデルロード完了")
print(f"  デバイス: {model.device}")
print(f"  埋め込み次元: {model.embedding_dim}")
print(f"  キャッシュ: {model.cache_dir}")

## 4. 基本動作テスト

### 4.1 画像エンコードテスト（ダミー画像）

In [ ]:
import numpy as np
from PIL import Image

# ダミー画像でエンコードテスト
dummy_img = Image.fromarray(np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8))
img_vec = model.encode_image(dummy_img)

print(f"画像ベクトル形状: {img_vec.shape}")  # (1, 768)
print(f"L2ノルム: {np.linalg.norm(img_vec):.6f}")  # ≈ 1.0
print("✅ 画像エンコード成功")

### 4.2 テキストエンコードテスト

In [ ]:
# テキストエンコードテスト
text_vec = model.encode_text("猫が寝ている写真")

print(f"テキストベクトル形状: {text_vec.shape}")  # (1, 768)
print(f"L2ノルム: {np.linalg.norm(text_vec):.6f}")  # ≈ 1.0
print("✅ テキストエンコード成功")

### 4.3 画像-テキスト類似度テスト

In [ ]:
# 複数テキストとダミー画像の類似度を計算
texts = ["犬", "猫", "車", "花", "海"]
text_vecs = model.encode_texts(texts)

# コサイン類似度（内積。ベクトルはL2正規化済み）
similarities = (img_vec @ text_vecs.T).squeeze()

print("ダミー画像と各テキストの類似度:")
for text, sim in sorted(zip(texts, similarities), key=lambda x: x[1], reverse=True):
    print(f"  {text}: {sim:.4f}")

## 5. ベクトルDBテスト

In [ ]:
from vector_db import VectorDB
import tempfile

# 一時ディレクトリでDBテスト
with tempfile.TemporaryDirectory() as tmp:
    db = VectorDB(db_dir=tmp, dim=768)
    print(f"初期件数: {db.count()}")

    # ダミーベクトル追加
    for i in range(5):
        vec = np.random.randn(768).astype(np.float32)
        vec = vec / np.linalg.norm(vec)
        db.add(vec, {"type": "image", "path": f"test_{i}.jpg", "filename": f"test_{i}.jpg"})

    print(f"追加後件数: {db.count()}")

    # 検索テスト
    query = np.random.randn(768).astype(np.float32)
    query = query / np.linalg.norm(query)
    results = db.search(query, top_k=3)

    print(f"\n検索結果 (top-3):")
    for r in results:
        print(f"  [{r['rank']}] score={r['score']:.4f} {r['metadata']['filename']}")

    # 保存・ロードテスト
    db.save()
    db2 = VectorDB(db_dir=tmp, dim=768)
    print(f"\nロード後件数: {db2.count()}")

print("✅ ベクトルDB動作OK")

## 6. 動画フレーム抽出テスト

In [ ]:
from video_processor import extract_frames, extract_frames_by_count, get_video_info

# 動画パスが設定されている場合のみテスト
if QUERY_VIDEO_PATH.exists():
    # メタ情報
    info = get_video_info(QUERY_VIDEO_PATH)
    print(f"動画情報:")
    for k, v in info.items():
        print(f"  {k}: {v}")

    # 等間隔で5フレーム抽出
    frames = extract_frames_by_count(QUERY_VIDEO_PATH, num_frames=5)
    print(f"\n抽出フレーム数: {len(frames)}")

    # 抽出したフレームを表示
    from IPython.display import display
    for i, frame in enumerate(frames[:3]):  # 最初の3フレームだけ表示
        print(f"\nフレーム {i+1}:")
        display(frame.resize((320, 180)))
else:
    print("⚠️ QUERY_VIDEO_PATH が未設定または存在しません。動画テストをスキップします。")
    print(f"  現在の値: {QUERY_VIDEO_PATH}")

## 7. 検索エンジン: ディレクトリ一括登録

指定ディレクトリ内の画像（jpg/png等）と動画（mp4等）を自動検出して一括登録します。
サブディレクトリも再帰的に探索されます。

In [ ]:
from search_engine import SearchEngine

# 検索エンジンを初期化
engine = SearchEngine(db_dir=DB_DIR, cache_dir=CACHE_DIR)
print(f"検索エンジン初期化完了")
print(f"  DB保存先: {engine.db.db_dir}")
print(f"  現在の登録数: {engine.db.count()}件")

In [ ]:
# ディレクトリ一括登録
if MEDIA_DIR.exists() and MEDIA_DIR.is_dir():
    print(f"登録対象ディレクトリ: {MEDIA_DIR}")

    # 対象ファイル数を事前確認
    from search_engine import IMAGE_EXTENSIONS, VIDEO_EXTENSIONS
    all_files = list(MEDIA_DIR.rglob("*"))
    img_count = sum(1 for f in all_files if f.suffix.lower() in IMAGE_EXTENSIONS)
    vid_count = sum(1 for f in all_files if f.suffix.lower() in VIDEO_EXTENSIONS)
    print(f"  画像: {img_count}件, 動画: {vid_count}件")

    # 登録実行（recursive=True でサブディレクトリも探索）
    indices = engine.register_directory(
        MEDIA_DIR,
        recursive=True,
        frame_interval=FRAME_INTERVAL_SEC,
    )
    print(f"\n✅ 登録完了: {len(indices)}件")
else:
    print("⚠️ MEDIA_DIR が未設定または存在しません。")
    print(f"  現在の値: {MEDIA_DIR}")
    print("  上の設定セルでパスを変更してください。")

In [ ]:
# DB統計表示
stats = engine.get_stats()
print(f"DB統計:")
print(f"  合計: {stats['total']}件")
print(f"  画像: {stats['images']}件")
print(f"  動画: {stats['videos']}件")

## 8. テキストで検索

In [ ]:
# テキストで検索
results = engine.search_by_text(TEXT_QUERY, top_k=5)

print(f"検索クエリ: 「{TEXT_QUERY}」")
print(f"ヒット件数: {len(results)}")
print("-" * 60)

from IPython.display import display

for r in results:
    meta = r["metadata"]
    print(f"\n[{r['rank']}] スコア: {r['score']:.4f} | {meta['type']} | {meta['filename']}")
    print(f"    パス: {meta['path']}")

    # 画像の場合はプレビュー表示
    if meta["type"] == "image" and Path(meta["path"]).exists():
        img = Image.open(meta["path"])
        display(img.resize((320, int(320 * img.height / img.width))))

    # 動画の場合は先頭フレームをプレビュー
    if meta["type"] == "video" and Path(meta["path"]).exists():
        preview_frames = extract_frames_by_count(meta["path"], num_frames=1)
        if preview_frames:
            display(preview_frames[0].resize((320, 180)))

## 9. 画像で検索

In [ ]:
# 画像で検索
if QUERY_IMAGE_PATH.exists():
    print(f"クエリ画像: {QUERY_IMAGE_PATH}")
    query_img = Image.open(QUERY_IMAGE_PATH)
    display(query_img.resize((320, int(320 * query_img.height / query_img.width))))

    results = engine.search_by_image(QUERY_IMAGE_PATH, top_k=5)

    print(f"\nヒット件数: {len(results)}")
    print("-" * 60)

    for r in results:
        meta = r["metadata"]
        print(f"\n[{r['rank']}] スコア: {r['score']:.4f} | {meta['type']} | {meta['filename']}")

        if meta["type"] == "image" and Path(meta["path"]).exists():
            img = Image.open(meta["path"])
            display(img.resize((320, int(320 * img.height / img.width))))

        if meta["type"] == "video" and Path(meta["path"]).exists():
            preview_frames = extract_frames_by_count(meta["path"], num_frames=1)
            if preview_frames:
                display(preview_frames[0].resize((320, 180)))
else:
    print("⚠️ QUERY_IMAGE_PATH が未設定または存在しません。画像検索をスキップします。")
    print(f"  現在の値: {QUERY_IMAGE_PATH}")

## 10. 動画で検索

In [ ]:
# 動画で検索
if QUERY_VIDEO_PATH.exists():
    print(f"クエリ動画: {QUERY_VIDEO_PATH}")

    # クエリ動画のプレビュー
    preview = extract_frames_by_count(QUERY_VIDEO_PATH, num_frames=1)
    if preview:
        display(preview[0].resize((320, 180)))

    results = engine.search_by_video(
        QUERY_VIDEO_PATH, top_k=5, frame_interval=FRAME_INTERVAL_SEC
    )

    print(f"\nヒット件数: {len(results)}")
    print("-" * 60)

    for r in results:
        meta = r["metadata"]
        print(f"\n[{r['rank']}] スコア: {r['score']:.4f} | {meta['type']} | {meta['filename']}")

        if meta["type"] == "image" and Path(meta["path"]).exists():
            img = Image.open(meta["path"])
            display(img.resize((320, int(320 * img.height / img.width))))

        if meta["type"] == "video" and Path(meta["path"]).exists():
            preview_frames = extract_frames_by_count(meta["path"], num_frames=1)
            if preview_frames:
                display(preview_frames[0].resize((320, 180)))
else:
    print("⚠️ QUERY_VIDEO_PATH が未設定または存在しません。動画検索をスキップします。")
    print(f"  現在の値: {QUERY_VIDEO_PATH}")

## 11. DB管理

In [ ]:
# 登録一覧を確認
entries = engine.db.list_entries()
print(f"登録エントリ一覧 ({len(entries)}件):")
for entry in entries[:20]:  # 最初の20件のみ表示
    meta = entry["metadata"]
    media_type = "画像" if meta["type"] == "image" else "動画"
    print(f"  [{entry['index']}] {media_type}: {meta['filename']}")

if len(entries) > 20:
    print(f"  ... 他 {len(entries) - 20}件")

In [ ]:
# DBをクリアする場合はこのセルを実行（通常はコメントアウト）
# engine.db.clear()
# print("DBをクリアしました")

## 12. CLIリファレンス

ターミナルからも利用できます:

```bash
# ディレクトリ一括登録（サブディレクトリも再帰的に探索）
uv run python clip-search-engine/run.py register --dir "D:\写真\旅行2024"

# サブディレクトリを探索しない場合
uv run python clip-search-engine/run.py register --dir "D:\写真" --no-recursive

# 画像を個別登録
uv run python clip-search-engine/run.py register --image "D:\写真\photo.jpg"

# 動画を個別登録（フレーム間隔指定）
uv run python clip-search-engine/run.py register --video "D:\動画\video.mp4" --frame-interval 5

# テキストで検索
uv run python clip-search-engine/run.py search --text "猫が寝ている"

# 画像で検索
uv run python clip-search-engine/run.py search --image "D:\写真\query.jpg"

# 動画で検索
uv run python clip-search-engine/run.py search --video "D:\動画\query.mp4" --top-k 10

# DB情報表示
uv run python clip-search-engine/run.py info --list-all

# DBクリア
uv run python clip-search-engine/run.py clear -y

# モデルキャッシュを指定（既存のキャッシュを再利用）
uv run python clip-search-engine/run.py --cache-dir "D:\models\clip" search --text "犬"
```

# CLIP Japanese v2 画像・動画検索エンジンを試す

## 目的
`line-corporation/clip-japanese-base-v2` を使った画像・動画検索エンジン（`clip-search-engine`）の動作確認を行う。

### 検証項目
1. **モデルの動作確認**: CLIP Japanese v2 で画像・テキストをベクトル化できるか
2. **ベクトルDBの動作確認**: FAISSによるベクトルの保存・検索が正しく動くか
3. **画像の登録と検索**: 擬似画像を登録し、テキスト・画像で検索できるか
4. **動画フレーム抽出**: OpenCVで動画フレームを10秒おきに抽出できるか
5. **ディレクトリ一括登録**: フォルダ内の画像を一括で登録・検索できるか

### アーキテクチャ概要
- **CLIPモデル**: `line-corporation/clip-japanese-base-v2`（768次元ベクトル）
- **ベクトルDB**: FAISS `IndexFlatIP`（コサイン類似度検索）
- **動画処理**: OpenCVで10秒おきにフレーム抽出 → 平均ベクトル化

## 1. 環境セットアップ

In [ ]:
import sys
import os
import tempfile
import logging
from pathlib import Path

import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

# clip-search-engine のモジュールをインポートできるようにパスを追加
PROJECT_ROOT = Path(os.getcwd()).parent if "notebooks" in os.getcwd() else Path(os.getcwd())
CLIP_ENGINE_DIR = PROJECT_ROOT / "clip-search-engine"
sys.path.insert(0, str(CLIP_ENGINE_DIR))

# ロギング設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# デバイス確認
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")
print(f"CLIP Engine Dir: {CLIP_ENGINE_DIR}")
print(f"プロジェクトルート: {PROJECT_ROOT}")

## 2. 擬似テストデータの生成

サニティチェック用に、異なるカテゴリを模した擬似画像を生成する。
- **赤い画像**: 赤系の色で塗りつぶし + テキスト
- **青い画像**: 青系の色で塗りつぶし + テキスト
- **緑の画像**: 緑系の色で塗りつぶし + テキスト
- **猫っぽい画像**: 猫の簡易な図形を描画
- **風景っぽい画像**: 空と地面の簡易描画

In [ ]:
# テスト用の擬似画像を生成する関数群

def create_colored_image(color: tuple[int, int, int], label: str, size: int = 224) -> Image.Image:
    """指定した色で塗りつぶした画像を生成する。"""
    img = Image.new("RGB", (size, size), color)
    draw = ImageDraw.Draw(img)
    # テキストをラベルとして中央に描画
    draw.text((size // 4, size // 2 - 10), label, fill=(255, 255, 255))
    return img


def create_cat_like_image(size: int = 224) -> Image.Image:
    """猫っぽい簡易図形を描画した画像を生成する。"""
    img = Image.new("RGB", (size, size), (200, 180, 150))
    draw = ImageDraw.Draw(img)
    # 顔（円）
    cx, cy = size // 2, size // 2
    r = size // 4
    draw.ellipse([cx - r, cy - r, cx + r, cy + r], fill=(180, 150, 120))
    # 耳（三角）
    draw.polygon([(cx - r, cy - r), (cx - r + 20, cy - r - 30), (cx - r + 40, cy - r)], fill=(160, 130, 100))
    draw.polygon([(cx + r - 40, cy - r), (cx + r - 20, cy - r - 30), (cx + r, cy - r)], fill=(160, 130, 100))
    # 目
    draw.ellipse([cx - 20, cy - 10, cx - 10, cy + 5], fill=(50, 200, 50))
    draw.ellipse([cx + 10, cy - 10, cx + 20, cy + 5], fill=(50, 200, 50))
    # 鼻
    draw.polygon([(cx - 5, cy + 15), (cx + 5, cy + 15), (cx, cy + 20)], fill=(255, 150, 150))
    return img


def create_landscape_image(size: int = 224) -> Image.Image:
    """風景っぽい簡易画像を生成する（空 + 地面 + 太陽）。"""
    img = Image.new("RGB", (size, size), (135, 206, 235))  # 空色
    draw = ImageDraw.Draw(img)
    # 地面（緑）
    draw.rectangle([0, size * 2 // 3, size, size], fill=(34, 139, 34))
    # 太陽
    draw.ellipse([size - 60, 20, size - 20, 60], fill=(255, 223, 0))
    # 雲
    draw.ellipse([30, 30, 80, 55], fill=(255, 255, 255))
    draw.ellipse([50, 25, 110, 55], fill=(255, 255, 255))
    draw.ellipse([80, 30, 130, 55], fill=(255, 255, 255))
    return img


def create_car_like_image(size: int = 224) -> Image.Image:
    """車っぽい簡易画像を生成する。"""
    img = Image.new("RGB", (size, size), (200, 200, 200))
    draw = ImageDraw.Draw(img)
    # ボディ
    draw.rectangle([40, 100, 184, 150], fill=(200, 0, 0))
    # 屋根
    draw.polygon([(70, 100), (90, 70), (150, 70), (170, 100)], fill=(180, 0, 0))
    # タイヤ
    draw.ellipse([55, 140, 85, 170], fill=(30, 30, 30))
    draw.ellipse([139, 140, 169, 170], fill=(30, 30, 30))
    # 窓
    draw.rectangle([95, 75, 120, 95], fill=(180, 220, 255))
    draw.rectangle([125, 75, 145, 95], fill=(180, 220, 255))
    return img


# 一時ディレクトリに画像を保存
test_dir = Path(tempfile.mkdtemp(prefix="clip_test_"))
print(f"テスト用ディレクトリ: {test_dir}")

# 各カテゴリの画像を生成して保存
test_images = {
    "red_image.jpg": create_colored_image((220, 30, 30), "RED"),
    "blue_image.jpg": create_colored_image((30, 30, 220), "BLUE"),
    "green_image.jpg": create_colored_image((30, 180, 30), "GREEN"),
    "cat_image.jpg": create_cat_like_image(),
    "landscape_image.jpg": create_landscape_image(),
    "car_image.jpg": create_car_like_image(),
}

for filename, img in test_images.items():
    img.save(test_dir / filename)

# 生成した画像を一覧表示
fig, axes = plt.subplots(1, len(test_images), figsize=(18, 3))
for ax, (name, img) in zip(axes, test_images.items()):
    ax.imshow(img)
    ax.set_title(name.replace("_image.jpg", ""))
    ax.axis("off")
plt.suptitle("生成した擬似テスト画像", fontsize=14)
plt.tight_layout()
plt.show()

print(f"✅ {len(test_images)}枚の擬似画像を生成しました")

## 3. CLIPモデルの動作確認

CLIP Japanese v2 モデルをロードし、画像とテキストのベクトル化が正しく動作するか確認する。

- 初回実行時はモデルのダウンロードが走るため、数分かかる場合がある
- モデルサイズ: 約200Mパラメータ

In [ ]:
from clip_model import CLIPJapaneseModel

# モデルをロード（初回はダウンロードが発生する）
clip_model = CLIPJapaneseModel(device=device)

print(f"モデルロード完了")
print(f"  埋め込み次元: {clip_model.embedding_dim}")
print(f"  デバイス: {clip_model.device}")

### 3.1 画像のベクトル化テスト

擬似画像をベクトル化し、ベクトルの形状と正規化を確認する。

In [ ]:
# 各画像をベクトル化して確認
print("画像のベクトル化テスト:")
print("-" * 50)

image_vectors = {}
for name, img in test_images.items():
    vec = clip_model.encode_image(img)
    image_vectors[name] = vec
    norm = np.linalg.norm(vec)
    print(f"  {name:25s} -> shape={vec.shape}, L2 norm={norm:.6f}")

print(f"\n✅ 全画像のベクトル化完了（{len(image_vectors)}枚）")
print(f"   各ベクトルはL2正規化済み（norm ≈ 1.0）")

### 3.2 テキストのベクトル化テスト

日本語テキストをベクトル化し、画像ベクトルとのコサイン類似度を計算する。

In [ ]:
# テスト用クエリテキスト
test_queries = ["赤い色", "青い色", "猫", "風景", "自動車", "緑の草原"]

# テキストをベクトル化
text_vectors = {}
for query in test_queries:
    vec = clip_model.encode_text(query)
    text_vectors[query] = vec

# 全テキスト × 全画像のコサイン類似度マトリクスを計算
# （正規化済みベクトル同士の内積 = コサイン類似度）
image_names = list(image_vectors.keys())
query_names = list(text_vectors.keys())

similarity_matrix = np.zeros((len(query_names), len(image_names)))
for i, q in enumerate(query_names):
    for j, img_name in enumerate(image_names):
        similarity_matrix[i, j] = float(text_vectors[q] @ image_vectors[img_name].T)

# ヒートマップで可視化
fig, ax = plt.subplots(figsize=(10, 6))
im = ax.imshow(similarity_matrix, cmap="YlOrRd", aspect="auto")
ax.set_xticks(range(len(image_names)))
ax.set_xticklabels([n.replace("_image.jpg", "") for n in image_names], rotation=45, ha="right")
ax.set_yticks(range(len(query_names)))
ax.set_yticklabels(query_names)
ax.set_xlabel("画像")
ax.set_ylabel("テキストクエリ")
ax.set_title("テキスト × 画像 コサイン類似度マトリクス")

# 各セルにスコアを表示
for i in range(len(query_names)):
    for j in range(len(image_names)):
        ax.text(j, i, f"{similarity_matrix[i, j]:.3f}",
                ha="center", va="center", fontsize=9,
                color="white" if similarity_matrix[i, j] > 0.25 else "black")

plt.colorbar(im, label="コサイン類似度")
plt.tight_layout()
plt.show()

print("✅ テキスト-画像間のコサイン類似度を計算・可視化しました")

## 4. ベクトルDBの動作確認

FAISSベクトルDBに画像ベクトルを登録し、検索が正しく動くか確認する。

In [ ]:
from vector_db import VectorDB

# テスト用の一時DBディレクトリ
test_db_dir = Path(tempfile.mkdtemp(prefix="clip_db_test_"))
print(f"テスト用DBディレクトリ: {test_db_dir}")

# ベクトルDBを初期化
db = VectorDB(db_dir=test_db_dir, dim=clip_model.embedding_dim)
print(f"DB初期化完了 (dim={clip_model.embedding_dim}, count={db.count()})")

# 画像ベクトルを登録
for name, vec in image_vectors.items():
    meta = {"type": "image", "filename": name, "path": str(test_dir / name)}
    db.add(vec.squeeze(0), meta)

db.save()
print(f"\n✅ {db.count()}件のベクトルを登録・保存しました")

# テキストで検索テスト
print("\n--- テキスト検索テスト ---")
for query in ["猫", "赤い色", "風景と空"]:
    query_vec = clip_model.encode_text(query)
    results = db.search(query_vec, top_k=3)
    print(f"\nクエリ: 「{query}」")
    for r in results:
        print(f"  [{r['rank']}] {r['metadata']['filename']:25s} (score: {r['score']:.4f})")

## 5. 検索エンジン（SearchEngine）の統合テスト

`SearchEngine` クラスを使って、画像の登録からテキスト・画像検索までの一連のフローを確認する。

In [ ]:
from search_engine import SearchEngine

# 新しいDBディレクトリで検索エンジンを初期化
engine_db_dir = Path(tempfile.mkdtemp(prefix="clip_engine_test_"))
engine = SearchEngine(db_dir=engine_db_dir, device=device)

print(f"検索エンジン初期化完了")
print(f"  DB保存先: {engine_db_dir}")
print(f"  初期登録数: {engine.get_stats()}")

### 5.1 画像の個別登録

In [ ]:
# 画像を1枚ずつ登録
for filename in test_images.keys():
    image_path = test_dir / filename
    idx = engine.register_image(image_path)
    print(f"  登録: {filename} -> ID={idx}")

# 統計を確認
stats = engine.get_stats()
print(f"\n✅ 登録完了: {stats}")

### 5.2 テキスト検索

日本語テキストで検索し、関連する画像が上位にランクされるか確認する。

In [ ]:
# テキスト検索を実行し、結果を可視化する関数
def visualize_search_results(results: list[dict], query_desc: str) -> None:
    """検索結果をサムネイル付きで可視化する。"""
    n = len(results)
    if n == 0:
        print("検索結果なし")
        return

    fig, axes = plt.subplots(1, n, figsize=(4 * n, 4))
    if n == 1:
        axes = [axes]

    fig.suptitle(f"検索クエリ: 「{query_desc}」", fontsize=14, fontweight="bold")

    for ax, r in zip(axes, results):
        meta = r["metadata"]
        img = Image.open(meta["path"])
        ax.imshow(img)
        ax.set_title(f"#{r['rank']} {meta['filename']}\nscore: {r['score']:.4f}", fontsize=10)
        ax.axis("off")

    plt.tight_layout()
    plt.show()


# テキスト検索テスト
search_queries = ["猫の顔", "赤い色", "空と太陽の風景", "車", "青い海"]

for query in search_queries:
    results = engine.search_by_text(query, top_k=3)
    visualize_search_results(results, query)
    print()

### 5.3 画像検索

登録画像の1つをクエリとして使い、類似画像を検索する。同一画像がスコア1.0で返ることを確認する。

In [ ]:
# 猫の画像をクエリとして類似画像を検索
query_image_path = test_dir / "cat_image.jpg"
results = engine.search_by_image(query_image_path, top_k=4)

# クエリ画像と結果を並べて表示
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

# クエリ画像
axes[0].imshow(Image.open(query_image_path))
axes[0].set_title("クエリ画像\n(cat_image)", fontsize=11, fontweight="bold", color="red")
axes[0].axis("off")

# 検索結果
for i, r in enumerate(results):
    meta = r["metadata"]
    axes[i + 1].imshow(Image.open(meta["path"]))
    axes[i + 1].set_title(f"#{r['rank']} {meta['filename']}\nscore: {r['score']:.4f}", fontsize=10)
    axes[i + 1].axis("off")

plt.suptitle("画像をクエリとした検索結果", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

# 同一画像のスコアが最大であることを確認
print(f"同一画像（cat_image.jpg）のスコア: {results[0]['score']:.6f}")
print(f"期待値: ≈ 1.0（同一画像なので完全一致）")

## 6. 動画フレーム抽出のテスト

OpenCVで擬似動画を生成し、10秒おきのフレーム抽出が正しく動作するか確認する。

In [ ]:
import cv2

# 擬似動画を生成する（30秒間、色が徐々に変化する動画）
video_path = test_dir / "test_video.mp4"
fps = 30
duration_sec = 35  # 35秒（10秒おきに4フレーム抽出されるはず）
width, height = 224, 224

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(video_path), fourcc, fps, (width, height))

total_frames = fps * duration_sec
for frame_idx in range(total_frames):
    t = frame_idx / total_frames  # 0.0 ~ 1.0

    # 時間に応じて色が変化する画像を生成
    r = int(255 * (1 - t))
    g = int(255 * t)
    b = int(128 + 127 * np.sin(t * 2 * np.pi))

    frame = np.full((height, width, 3), [b, g, r], dtype=np.uint8)  # BGR

    # フレーム番号と経過時間を表示
    time_sec = frame_idx / fps
    cv2.putText(frame, f"t={time_sec:.1f}s", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    writer.write(frame)

writer.release()
print(f"✅ 擬似動画を生成しました: {video_path}")
print(f"   fps={fps}, duration={duration_sec}s, total_frames={total_frames}")

In [ ]:
from video_processor import extract_frames, get_video_info

# 動画メタ情報を確認
info = get_video_info(video_path)
print("動画メタ情報:")
for key, val in info.items():
    print(f"  {key}: {val}")

# フレーム抽出（10秒おき）
frames = extract_frames(video_path, interval_sec=10.0)
print(f"\n抽出されたフレーム数: {len(frames)}")
print(f"期待値: {int(duration_sec / 10) + 1}フレーム（0s, 10s, 20s, 30s）")

# 抽出されたフレームを表示
fig, axes = plt.subplots(1, len(frames), figsize=(5 * len(frames), 4))
if len(frames) == 1:
    axes = [axes]

for i, (ax, frame) in enumerate(zip(axes, frames)):
    ax.imshow(frame)
    ax.set_title(f"Frame {i}\n(t ≈ {i * 10}s)", fontsize=11)
    ax.axis("off")

plt.suptitle("10秒おきに抽出されたフレーム", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

## 7. 動画の登録と検索

擬似動画をDBに登録し、テキスト・画像で検索して動画もヒットすることを確認する。

動画の登録処理:
1. 10秒おきにフレームを抽出
2. 各フレームをCLIPで特徴量化
3. 全フレームの特徴量を**平均化**して1つのベクトルに
4. L2正規化してDBに保存

In [ ]:
# 動画をDBに登録
video_idx = engine.register_video(video_path, frame_interval=10.0)
print(f"✅ 動画を登録しました (ID={video_idx})")

# 登録統計を確認
stats = engine.get_stats()
print(f"DB統計: {stats}")

# DB内の全エントリを一覧表示
entries = engine.db.list_entries()
print("\n--- 登録済みエントリ一覧 ---")
for entry in entries:
    meta = entry["metadata"]
    media_type = "🖼️ 画像" if meta["type"] == "image" else "🎬 動画"
    extra = f" (frames={meta.get('num_frames', '')})" if meta["type"] == "video" else ""
    print(f"  [{entry['index']}] {media_type} {meta['filename']}{extra}")

In [ ]:
# テキスト検索で動画もヒットするか確認
print("--- 画像＋動画が混在するDBでのテキスト検索 ---\n")

mixed_queries = ["色が変わる映像", "猫", "赤い色", "風景"]
for query in mixed_queries:
    results = engine.search_by_text(query, top_k=3)
    print(f"クエリ: 「{query}」")
    for r in results:
        meta = r["metadata"]
        icon = "🖼️" if meta["type"] == "image" else "🎬"
        print(f"  [{r['rank']}] {icon} {meta['filename']:25s} score={r['score']:.4f}")
    print()

## 8. ディレクトリ一括登録のテスト

新しいディレクトリに追加画像を生成し、`register_directory()` で一括登録する。

In [ ]:
# 新しい一括登録用ディレクトリを作成
batch_dir = Path(tempfile.mkdtemp(prefix="clip_batch_"))

# 様々なパターンの画像を追加生成
batch_images = {
    "yellow_circle.png": Image.new("RGB", (224, 224), (255, 255, 50)),
    "dark_image.png": Image.new("RGB", (224, 224), (20, 20, 30)),
    "pink_image.jpg": Image.new("RGB", (224, 224), (255, 150, 180)),
}

# 黄色い円の画像にはちゃんと円を描画
draw = ImageDraw.Draw(batch_images["yellow_circle.png"])
draw.ellipse([50, 50, 174, 174], fill=(255, 220, 0), outline=(200, 150, 0), width=3)

for filename, img in batch_images.items():
    img.save(batch_dir / filename)

print(f"一括登録用ディレクトリ: {batch_dir}")
print(f"ファイル数: {len(batch_images)}")

# 新しいエンジンでディレクトリ一括登録テスト
batch_db_dir = Path(tempfile.mkdtemp(prefix="clip_batch_db_"))
batch_engine = SearchEngine(db_dir=batch_db_dir, device=device)

indices = batch_engine.register_directory(batch_dir)
print(f"\n✅ 一括登録完了: {len(indices)}件")
print(f"DB統計: {batch_engine.get_stats()}")

# 登録内容を確認
entries = batch_engine.db.list_entries()
for entry in entries:
    print(f"  [{entry['index']}] {entry['metadata']['filename']}")

## 9. DBの永続化と再ロード

DBを保存した後、新しいVectorDBインスタンスで再ロードし、データが保持されていることを確認する。

In [ ]:
# engineのDBは register_image/video 時に自動保存されている
# 別のVectorDBインスタンスで再ロードして確認
print(f"保存先: {engine_db_dir}")
print(f"保存前の件数: {engine.db.count()}")

# 新しいVectorDBインスタンスで同じディレクトリからロード
reloaded_db = VectorDB(db_dir=engine_db_dir, dim=clip_model.embedding_dim)
print(f"再ロード後の件数: {reloaded_db.count()}")

# 再ロードしたDBで検索テスト
query_vec = clip_model.encode_text("猫の顔")
results = reloaded_db.search(query_vec, top_k=3)
print(f"\n再ロードDBでの検索テスト（クエリ: 「猫の顔」）:")
for r in results:
    meta = r["metadata"]
    print(f"  [{r['rank']}] {meta['filename']:25s} score={r['score']:.4f}")

# 件数の一致を確認
assert engine.db.count() == reloaded_db.count(), "保存前後で件数が一致しません"
print(f"\n✅ DB永続化テスト通過（{reloaded_db.count()}件）")

## 10. 実際の画像で試す（任意）

ここから先は、実際の画像や動画をパスで指定して登録・検索を試すことができる。
自分の画像フォルダのパスを `YOUR_IMAGE_DIR` に設定して実行してください。

In [ ]:
# ======================================
# ここを編集して自分の画像で試す
# ======================================

# 本番用のベクトルDBディレクトリ（clip-search-engine/db/ がデフォルト）
PRODUCTION_DB_DIR = CLIP_ENGINE_DIR / "db"

# 登録したい画像・動画のパスやディレクトリ
# YOUR_IMAGE_DIR = Path(r"C:\path\to\your\images")
# YOUR_VIDEO_PATH = Path(r"C:\path\to\your\video.mp4")

# --- 実行例（コメントを外して使用） ---

# 本番DBを使う検索エンジンを初期化
# prod_engine = SearchEngine(db_dir=PRODUCTION_DB_DIR, device=device)

# ディレクトリ一括登録
# prod_engine.register_directory(YOUR_IMAGE_DIR)

# 動画登録
# prod_engine.register_video(YOUR_VIDEO_PATH, frame_interval=10.0)

# テキスト検索
# results = prod_engine.search_by_text("猫が寝ている", top_k=5)
# for r in results:
#     print(f"[{r['rank']}] {r['metadata']['filename']} (score={r['score']:.4f})")

print("💡 上のコメントを解除して、自分の画像・動画で試してください")

## 11. CLIの使い方

ノートブック外から、ターミナルでCLIを使うこともできる。

```bash
# プロジェクトルートから実行

# 画像を登録
uv run python clip-search-engine/run.py register --image path/to/image.jpg

# 動画を登録（10秒おきにフレーム抽出）
uv run python clip-search-engine/run.py register --video path/to/video.mp4

# ディレクトリ一括登録
uv run python clip-search-engine/run.py register --dir path/to/media/

# テキストで検索
uv run python clip-search-engine/run.py search --text "猫が寝ている"

# 画像で検索
uv run python clip-search-engine/run.py search --image path/to/query.jpg

# DB情報を表示
uv run python clip-search-engine/run.py info --list-all

# DBをクリア
uv run python clip-search-engine/run.py clear -y
```